In [1]:
#Import Statements
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, losses

import pandas as pd

from matplotlib import pyplot as plt

import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score

import keras
from keras import layers


In [2]:
# #ECFP featuriser - splitting the training dataset into smaller datasets
# #THIS WORKS FOR CONV MOL FEAT
# tasks, datasets, transformers = dc.molnet.load_tox21(featurizer=dc.feat.ConvMolFeaturizer(),
#     splitter='random',
#     save_dir=r'C:\Users\ym20201\Documents\Datasets',
#     data_dir=r'C:\Users\ym20201\Documents\Datasets')

#

In [44]:
#ECFP featuriser - splitting the training dataset into smaller datasets
tasks, datasets, transformers = dc.molnet.load_tox21(
    featurizer=dc.feat.CircularFingerprint(),
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

# featurizer = dc.feat.CircularFingerprint(size=2048, radius=4)
splitter = dc.splits.RandomSplitter()

train_data, valid_data,test_data = splitter.train_valid_test_split(
    datasets[0], 
    frac_train = 0.8, frac_valid = 0.08, frac_test = 0.12)

# #Featurisation using Featurizer
# train_feat = featurizer.featurize(train_data.ids)
# valid_feat = featurizer.featurize(valid_data.ids)
# test_feat = featurizer.featurize(test_data.ids)



In [45]:
 train_data, valid_data, test_data = datasets

In [4]:
# train = tf.convert_to_tensor(train_data.ids)

In [5]:
train_data

<DiskDataset X.shape: (5011, 2048), y.shape: (5011, 12), w.shape: (5011, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>

In [6]:
valid_data

<DiskDataset X.shape: (501, 2048), y.shape: (501, 12), w.shape: (501, 12), ids: ['O=C(O)c1ccc(O)cc1' 'OCCSSCCO' 'Cc1ccc(S(N)(=O)=O)cc1' ...
 'C=C/C(C)=C/C/C=C(\\C)CCC=C(C)C' 'CC(=O)Oc1cccc(O)c1'
 'CC(C)=CCCC(C)=CC(=O)O'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>

In [10]:
len(valid_data.X[3])

2048

In [52]:
ECFP_model = dc.models.MultitaskClassifier(
    n_tasks = len(tasks),
    n_features = len(valid_data.X[3]),
    layer_sizes=[1000], # how many layers and how many units in each layer to have.
    weight_init_stddevs=0.02, 
    bias_init_consts=1.0,
    dropouts=[0.5, 0.0],
    activation_fns=['relu', 'relu'],
    n_classes=12,
    learning_rate=0.01,
    batch_size=100)

In [53]:
valid_data

<DiskDataset X.shape: (783, 2048), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
 'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
 'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
 'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
 'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
 'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>

In [71]:
patience =1
nb_epochs = 10
# metric1 = dc.metrics.Metric(dc.metrics.pearson_r2_score)
# metric2 = dc.metrics.Metric(dc.metrics.mae_score)
# metrics = [metric1, metric2]

roc_metric = dc.metrics.Metric(dc.metrics.roc_auc_score) #doesnt work
acc_metric = dc.metrics.Metric(dc.metrics.balanced_accuracy_score)
topk_acc_metric = dc.metrics.Metric(dc.metrics.top_k_accuracy_score)

callback = dc.models.ValidationCallback(
            valid_data,
            patience,
            metrics=metric)

AttributeError: module 'deepchem.metrics' has no attribute 'top_k_accuracy_score'

In [59]:
ECFP_model.fit(train_data,
              nb_epoch=10)

0.38288240432739257

In [51]:
train_data.y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
ECFP_model.evaluate(train_data, [acc_metric], transformers)

{'balanced_accuracy_score': 0.9708232989079111}

In [ ]:
# model = dc.models.GraphConvModel(len(tasks),
#                                 batch_size=32,
#                                  mode='classification')

# # fits the model
# model.fit(train_data, nb_epoch=10)
# # sets R2 as the metric
# metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
# #tests and prints out the model
# print('training set score:', model.evaluate(train_data, [metric], transformers))
# # print('test set score:', model.evaluate(test_data, [metric], transformers))

In [ ]:
# model = Sequential()
# model.add(layers.Conv2D(filters=32,
#                           kernel_size=3,
#                           activation='relu',
#                           dilation_rate=2,
#                              input_shape=(80, 80, 1)))
# model.add(layers.MaxPooling2D(pool_size=(2,2)))
# #dropout to avoid overfitting
# model.add(layers.Dense(50, activation='relu'))
# model.add(layers.Dense(1))

# model.build()

In [ ]:
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.summary()

In [ ]:
# model.fit(x=train_feat, y=train_feat,
#                 epochs=10,
#                 batch_size=512,
#                 shuffle=True,
#                validation_data=(valid_feat,valid_feat))